# Case Study 3: Healthcare Patient Readmission Prediction

## HealthFirst Medical Center - 30-Day Readmission Risk with Fairness-Aware AutoML

This notebook demonstrates the complete implementation from Case Study 3, showing how to:

1. **Generate Synthetic Healthcare Data** - Create realistic EHR data with clinical complexity
2. **HIPAA-Compliant Data Pipeline** - Implement Safe Harbor de-identification
3. **Multimodal Learning** - Combine structured EHR data with clinical text
4. **Fairness-Aware Training** - Build models that are fair across demographic groups
5. **Clinical Evaluation** - Measure both performance and fairness metrics

### Key Results from Case Study
- **Performance**: 0.73 AUC (vs. 0.61 LACE index baseline)
- **Fairness**: <3pp AUC gap across race, age, and insurance groups
- **Impact**: 18% reduction in readmissions (17.8% → 14.6%)
- **ROI**: $79.1M annual value, 720% ROI

## Data Generation & Reproducibility

**Important**: This notebook uses **synthetic data** generated programmatically. All data is simulated to demonstrate the methodology while maintaining HIPAA compliance. The notebook is **fully reproducible** - no external data files required.

### Synthetic Data Characteristics
- **10,000 patient admissions** with realistic clinical patterns
- **23% missing data** across various fields (matching real-world EHR incompleteness)
- **Protected demographics**: Race, age groups, insurance type for fairness testing
- **Clinical complexity**: Charlson/Elixhauser comorbidities, labs, vitals, medications
- **Discharge summaries**: Synthetic clinical text with NLP-extractable patterns
- **Realistic distributions**: Readmission rate ~15-18%, clinical risk factors from literature

### Expected Performance
- **Overall AUC**: 0.70-0.74 (multimodal ensemble)
- **Fairness Metrics**: <5pp gaps in AUC, TPR, FPR across demographic groups
- **Calibration**: Brier score 0.12-0.15, calibration slope 0.95-1.05

In [ ]:
# Install required packages
import sys
!pip install -q 'autogluon>=1.5.0' scikit-learn pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, brier_score_loss
from sklearn.calibration import calibration_curve
import json

warnings.filterwarnings('ignore')
np.random.seed(42)

print("Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 1. Synthetic EHR Data Generator

This class generates realistic electronic health record data with:
- **Demographics**: Age, gender, race, insurance with realistic missingness
- **Comorbidities**: Charlson and Elixhauser conditions
- **Clinical data**: Lab values, vital signs, procedures, medications
- **Utilization history**: Prior ED visits, hospitalizations, readmissions
- **Social determinants**: Housing status, food insecurity, transportation barriers
- **Clinical notes**: Synthetic discharge summaries with medical terminology
- **Readmission labels**: 30-day readmission with realistic risk factors

In [ ]:
class HealthcareDataGenerator:
    """
    Generates synthetic EHR data for patient readmission prediction.

    Simulates realistic patterns:
    - Comorbidity correlations (diabetes → CKD, COPD → heart failure)
    - Age-related risk (older patients → more comorbidities)
    - Social determinants impact (housing instability → higher risk)
    - Missing data patterns (23% incomplete records)
    """

    def __init__(self, n_patients=10000, random_state=42):
        self.n_patients = n_patients
        self.random_state = random_state
        np.random.seed(random_state)

        # Clinical note templates for synthetic discharge summaries
        self.note_templates = [
            "Patient admitted with {condition}. Medical history significant for {comorbidity}. {treatment} initiated. Discharge planning completed with follow-up arranged.",
            "Hospitalization for {condition}. Past medical history includes {comorbidity}. Clinical course complicated by {complication}. {treatment} provided. Patient stable at discharge.",
            "Acute {condition} requiring admission. Known {comorbidity}. Treatment with {treatment}. Care coordination arranged for outpatient management.",
            "Admitted for exacerbation of {condition}. Chronic {comorbidity} managed during stay. {treatment} administered. Social work consulted for {social_concern}.",
            "Emergency admission for {condition}. Significant history of {comorbidity}. {treatment} protocol followed. Discharge with home health services."
        ]

        self.conditions = [
            'heart failure exacerbation', 'COPD exacerbation', 'pneumonia',
            'sepsis', 'acute kidney injury', 'diabetic ketoacidosis',
            'stroke', 'myocardial infarction', 'GI bleeding', 'cellulitis'
        ]

        self.comorbidities = [
            'diabetes mellitus', 'hypertension', 'chronic kidney disease',
            'coronary artery disease', 'COPD', 'heart failure',
            'atrial fibrillation', 'depression', 'dementia', 'cancer'
        ]

        self.treatments = [
            'IV antibiotics', 'diuretics', 'insulin therapy',
            'anticoagulation', 'oxygen therapy', 'IV fluids',
            'vasopressors', 'bronchodilators', 'pain management'
        ]

        self.complications = [
            'fluid overload', 'acute renal failure', 'hypotension',
            'hyperglycemia', 'delirium', 'aspiration'
        ]

        self.social_concerns = [
            'housing instability', 'medication access', 'transportation barriers',
            'caregiver support', 'food insecurity'
        ]

    def generate_demographics(self):
        """Generate patient demographics with realistic distributions."""
        data = {}

        # Age: Skewed toward older adults (hospital patients)
        data['age'] = np.random.gamma(shape=8, scale=8, size=self.n_patients).astype(int)
        data['age'] = np.clip(data['age'], 18, 95)

        # Age groups for fairness analysis
        data['age_group'] = pd.cut(data['age'], bins=[0, 45, 65, 75, 100],
                                   labels=['18-44', '45-64', '65-74', '75+'])

        # Gender (balanced)
        data['gender'] = np.random.choice(['M', 'F'], size=self.n_patients, p=[0.48, 0.52])

        # Race with realistic US hospital distribution
        data['race'] = np.random.choice(
            ['White', 'Black', 'Hispanic', 'Asian', 'Other'],
            size=self.n_patients,
            p=[0.55, 0.20, 0.15, 0.07, 0.03]
        )

        # Insurance type (protected attribute for fairness)
        # Age influences insurance (65+ mostly Medicare)
        insurance_probs = []
        for age in data['age']:
            if age >= 65:
                insurance_probs.append([0.75, 0.15, 0.05, 0.05])  # Mostly Medicare
            else:
                insurance_probs.append([0.10, 0.55, 0.25, 0.10])  # Mostly Commercial/Medicaid

        data['insurance'] = np.array([
            np.random.choice(['Medicare', 'Commercial', 'Medicaid', 'Uninsured'], p=p)
            for p in insurance_probs
        ])

        # Marital status
        data['marital_status'] = np.random.choice(
            ['Married', 'Single', 'Widowed', 'Divorced'],
            size=self.n_patients,
            p=[0.45, 0.25, 0.18, 0.12]
        )

        return pd.DataFrame(data)

    def generate_comorbidities(self, demographics):
        """Generate Charlson/Elixhauser comorbidities with age correlation."""
        n = len(demographics)

        # Base probabilities increase with age
        age_factor = (demographics['age'] / 100) ** 2

        comorbidities = {}

        # Charlson Comorbidity Index components
        comorbidities['diabetes'] = (np.random.random(n) < (0.15 + 0.25 * age_factor)).astype(int)
        comorbidities['chf'] = (np.random.random(n) < (0.10 + 0.30 * age_factor)).astype(int)
        comorbidities['copd'] = (np.random.random(n) < (0.12 + 0.20 * age_factor)).astype(int)
        comorbidities['ckd'] = (np.random.random(n) < (0.08 + 0.25 * age_factor)).astype(int)
        comorbidities['cancer'] = (np.random.random(n) < (0.05 + 0.15 * age_factor)).astype(int)
        comorbidities['dementia'] = (np.random.random(n) < (0.02 + 0.20 * age_factor)).astype(int)

        # Comorbidity interactions (diabetes increases CKD risk)
        diabetes_ckd_interaction = comorbidities['diabetes'] * (np.random.random(n) < 0.3)
        comorbidities['ckd'] = np.maximum(comorbidities['ckd'], diabetes_ckd_interaction.astype(int))

        # COPD increases CHF risk
        copd_chf_interaction = comorbidities['copd'] * (np.random.random(n) < 0.25)
        comorbidities['chf'] = np.maximum(comorbidities['chf'], copd_chf_interaction.astype(int))

        # Elixhauser components
        comorbidities['hypertension'] = (np.random.random(n) < (0.40 + 0.30 * age_factor)).astype(int)
        comorbidities['depression'] = (np.random.random(n) < 0.18).astype(int)
        comorbidities['obesity'] = (np.random.random(n) < 0.35).astype(int)
        comorbidities['anemia'] = (np.random.random(n) < (0.10 + 0.20 * age_factor)).astype(int)

        # Calculate Charlson Comorbidity Index (CCI)
        cci_scores = (
            comorbidities['diabetes'] +
            2 * comorbidities['chf'] +
            comorbidities['copd'] +
            2 * comorbidities['ckd'] +
            2 * comorbidities['cancer'] +
            comorbidities['dementia']
        )
        comorbidities['charlson_score'] = cci_scores

        # Elixhauser score (count of conditions)
        elixhauser_count = sum([
            comorbidities['hypertension'],
            comorbidities['diabetes'],
            comorbidities['chf'],
            comorbidities['copd'],
            comorbidities['ckd'],
            comorbidities['cancer'],
            comorbidities['depression'],
            comorbidities['obesity'],
            comorbidities['anemia']
        ])
        comorbidities['elixhauser_score'] = elixhauser_count

        return pd.DataFrame(comorbidities)

    def generate_clinical_data(self, demographics, comorbidities):
        """Generate lab values, vitals, and clinical measurements."""
        n = len(demographics)

        clinical = {}

        # Lab values (abnormal if comorbidities present)
        # Creatinine (higher with CKD, diabetes)
        ckd_factor = 1 + 2 * comorbidities['ckd']
        clinical['creatinine'] = np.random.gamma(2, 0.4, n) * ckd_factor
        clinical['creatinine'] = np.clip(clinical['creatinine'], 0.5, 8.0)

        # Hemoglobin (lower with anemia, CKD)
        anemia_factor = comorbidities['anemia'] + 0.5 * comorbidities['ckd']
        clinical['hemoglobin'] = 14 - 3 * anemia_factor + np.random.normal(0, 1.5, n)
        clinical['hemoglobin'] = np.clip(clinical['hemoglobin'], 6, 18)

        # BNP (brain natriuretic peptide - elevated in CHF)
        chf_factor = 1 + 20 * comorbidities['chf']
        clinical['bnp'] = np.random.gamma(2, 50, n) * chf_factor
        clinical['bnp'] = np.clip(clinical['bnp'], 10, 5000)

        # HbA1c (elevated in diabetes)
        diabetes_factor = 1 + 1.5 * comorbidities['diabetes']
        clinical['hba1c'] = 5.5 * diabetes_factor + np.random.normal(0, 0.8, n)
        clinical['hba1c'] = np.clip(clinical['hba1c'], 4.5, 14.0)

        # Vital signs
        # Blood pressure (higher with hypertension)
        htn_factor = 20 * comorbidities['hypertension']
        clinical['systolic_bp'] = 120 + htn_factor + np.random.normal(0, 15, n)
        clinical['systolic_bp'] = np.clip(clinical['systolic_bp'], 80, 200)

        clinical['diastolic_bp'] = 75 + 0.4 * htn_factor + np.random.normal(0, 10, n)
        clinical['diastolic_bp'] = np.clip(clinical['diastolic_bp'], 50, 120)

        # Heart rate (higher with CHF, COPD)
        hr_factor = 15 * (comorbidities['chf'] + comorbidities['copd'])
        clinical['heart_rate'] = 75 + hr_factor + np.random.normal(0, 12, n)
        clinical['heart_rate'] = np.clip(clinical['heart_rate'], 50, 140)

        # Respiratory rate (higher with COPD, CHF)
        rr_factor = 4 * (comorbidities['copd'] + comorbidities['chf'])
        clinical['respiratory_rate'] = 16 + rr_factor + np.random.normal(0, 3, n)
        clinical['respiratory_rate'] = np.clip(clinical['respiratory_rate'], 10, 35)

        # Oxygen saturation (lower with COPD)
        o2_factor = 5 * comorbidities['copd']
        clinical['oxygen_saturation'] = 98 - o2_factor + np.random.normal(0, 2, n)
        clinical['oxygen_saturation'] = np.clip(clinical['oxygen_saturation'], 85, 100)

        # BMI (obesity flag)
        obesity_factor = 10 * comorbidities['obesity']
        clinical['bmi'] = 25 + obesity_factor + np.random.normal(0, 4, n)
        clinical['bmi'] = np.clip(clinical['bmi'], 16, 50)

        return pd.DataFrame(clinical)

    def generate_utilization_history(self, demographics, comorbidities):
        """Generate prior healthcare utilization (ED visits, hospitalizations)."""
        n = len(demographics)

        # Utilization increases with comorbidity burden and age
        risk_factor = (
            0.3 * comorbidities['charlson_score'] +
            0.2 * (demographics['age'] / 100)
        )

        utilization = {}

        # ED visits in past year (Poisson distribution)
        utilization['ed_visits_1yr'] = np.random.poisson(lam=risk_factor * 2, size=n)
        utilization['ed_visits_1yr'] = np.clip(utilization['ed_visits_1yr'], 0, 20)

        # Hospitalizations in past year
        utilization['hospitalizations_1yr'] = np.random.poisson(lam=risk_factor * 1.5, size=n)
        utilization['hospitalizations_1yr'] = np.clip(utilization['hospitalizations_1yr'], 0, 10)

        # Prior readmissions (strong predictor of future readmission)
        utilization['prior_readmissions'] = np.random.poisson(lam=risk_factor * 0.8, size=n)
        utilization['prior_readmissions'] = np.clip(utilization['prior_readmissions'], 0, 5)

        # Days since last hospitalization
        # Patients with recent hospitalization at higher risk
        had_recent_hospitalization = utilization['hospitalizations_1yr'] > 0
        utilization['days_since_last_hosp'] = np.where(
            had_recent_hospitalization,
            np.random.exponential(scale=60, size=n),
            np.nan
        )
        utilization['days_since_last_hosp'] = np.clip(utilization['days_since_last_hosp'], 1, 365)

        # Primary care engagement (protective factor)
        # Fewer PCP visits = higher risk
        utilization['pcp_visits_1yr'] = np.random.poisson(lam=3 - risk_factor, size=n)
        utilization['pcp_visits_1yr'] = np.clip(utilization['pcp_visits_1yr'], 0, 12)

        return pd.DataFrame(utilization)

    def generate_current_encounter(self, demographics, comorbidities, utilization):
        """Generate current hospitalization details."""
        n = len(demographics)

        encounter = {}

        # Admission source
        encounter['admission_source'] = np.random.choice(
            ['Emergency', 'Transfer', 'Elective', 'Clinic'],
            size=n,
            p=[0.65, 0.15, 0.12, 0.08]
        )

        # Length of stay (longer LOS = higher readmission risk up to a point)
        comorbidity_factor = 1 + 0.5 * comorbidities['charlson_score']
        encounter['length_of_stay'] = np.random.gamma(2, 2 * comorbidity_factor, n)
        encounter['length_of_stay'] = np.clip(encounter['length_of_stay'], 1, 30)

        # ICU admission (high risk)
        icu_prob = 0.15 + 0.05 * comorbidities['charlson_score']
        encounter['icu_admission'] = (np.random.random(n) < icu_prob).astype(int)

        # ICU length of stay (if admitted to ICU)
        encounter['icu_days'] = np.where(
            encounter['icu_admission'] == 1,
            np.random.gamma(2, 2, n),
            0
        )
        encounter['icu_days'] = np.clip(encounter['icu_days'], 0, 15)

        # Number of procedures
        encounter['num_procedures'] = np.random.poisson(lam=1 + 0.3 * comorbidities['charlson_score'], size=n)
        encounter['num_procedures'] = np.clip(encounter['num_procedures'], 0, 10)

        # Number of medications at discharge
        encounter['num_medications'] = np.random.poisson(lam=8 + comorbidities['elixhauser_score'], size=n)
        encounter['num_medications'] = np.clip(encounter['num_medications'], 0, 25)

        # Medication changes (addition/removal increases complexity)
        encounter['medication_changes'] = np.random.poisson(lam=2 + 0.5 * comorbidities['charlson_score'], size=n)
        encounter['medication_changes'] = np.clip(encounter['medication_changes'], 0, 15)

        # Discharge disposition
        # Higher risk patients → SNF/Rehab; Very low risk → Home
        discharge_probs = []
        for i in range(n):
            cci = comorbidities['charlson_score'].iloc[i]
            if cci >= 5:
                discharge_probs.append([0.30, 0.35, 0.25, 0.10])  # Higher SNF/Rehab
            elif cci >= 3:
                discharge_probs.append([0.50, 0.25, 0.15, 0.10])
            else:
                discharge_probs.append([0.70, 0.15, 0.10, 0.05])  # Mostly home

        encounter['discharge_disposition'] = np.array([
            np.random.choice(['Home', 'SNF', 'Rehab', 'Home_Health'], p=p)
            for p in discharge_probs
        ])

        # Follow-up appointment scheduled (protective)
        encounter['followup_scheduled'] = (np.random.random(n) < 0.75).astype(int)

        # Days until follow-up (7-day follow-up = lower risk)
        encounter['days_to_followup'] = np.where(
            encounter['followup_scheduled'] == 1,
            np.random.choice([3, 7, 14, 30], size=n, p=[0.15, 0.45, 0.30, 0.10]),
            np.nan
        )

        return pd.DataFrame(encounter)

    def generate_social_determinants(self, demographics):
        """Generate social determinants of health (SDOH) data."""
        n = len(demographics)

        sdoh = {}

        # Housing status (instability = high risk)
        # Higher in Medicaid/Uninsured
        housing_risk = np.where(
            demographics['insurance'].isin(['Medicaid', 'Uninsured']),
            0.35, 0.10
        )
        sdoh['housing_unstable'] = (np.random.random(n) < housing_risk).astype(int)

        # Food insecurity
        food_risk = np.where(
            demographics['insurance'].isin(['Medicaid', 'Uninsured']),
            0.30, 0.08
        )
        sdoh['food_insecure'] = (np.random.random(n) < food_risk).astype(int)

        # Transportation barriers
        transport_risk = np.where(
            demographics['insurance'].isin(['Medicaid', 'Uninsured']),
            0.25, 0.06
        )
        sdoh['transportation_barrier'] = (np.random.random(n) < transport_risk).astype(int)

        # Caregiver support (protective, especially for older adults)
        # Married + older = more likely to have caregiver
        caregiver_prob = np.where(
            (demographics['marital_status'] == 'Married') & (demographics['age'] >= 65),
            0.75, 0.40
        )
        sdoh['has_caregiver'] = (np.random.random(n) < caregiver_prob).astype(int)

        # Area Deprivation Index (ADI) - neighborhood disadvantage (1-100)
        # Correlated with insurance type
        adi_base = np.where(
            demographics['insurance'] == 'Commercial', 30,
            np.where(demographics['insurance'] == 'Medicare', 50,
                    np.where(demographics['insurance'] == 'Medicaid', 70, 80))
        )
        sdoh['adi_score'] = adi_base + np.random.normal(0, 15, n)
        sdoh['adi_score'] = np.clip(sdoh['adi_score'], 1, 100)

        return pd.DataFrame(sdoh)

    def generate_discharge_summary(self, i, demographics, comorbidities, encounter):
        """Generate synthetic clinical discharge summary text."""
        # Select template and fill with patient-specific details
        template = np.random.choice(self.note_templates)

        # Choose condition based on comorbidities
        if comorbidities['chf'].iloc[i] and np.random.random() < 0.4:
            condition = 'heart failure exacerbation'
        elif comorbidities['copd'].iloc[i] and np.random.random() < 0.4:
            condition = 'COPD exacerbation'
        elif comorbidities['diabetes'].iloc[i] and np.random.random() < 0.2:
            condition = 'diabetic ketoacidosis'
        else:
            condition = np.random.choice(self.conditions)

        # Select comorbidity to mention
        active_comorbidities = [
            com for com in self.comorbidities
            if com in comorbidities.columns and comorbidities[com].iloc[i] == 1
        ]

        if active_comorbidities:
            comorbidity = np.random.choice(active_comorbidities)
        else:
            comorbidity = np.random.choice(self.comorbidities)

        treatment = np.random.choice(self.treatments)
        complication = np.random.choice(self.complications)
        social_concern = np.random.choice(self.social_concerns)

        # Fill template
        note = template.format(
            condition=condition,
            comorbidity=comorbidity,
            treatment=treatment,
            complication=complication,
            social_concern=social_concern
        )

        # Add age and gender context
        age = demographics['age'].iloc[i]
        gender = 'male' if demographics['gender'].iloc[i] == 'M' else 'female'
        prefix = f"{age}-year-old {gender}. "

        return prefix + note

    def calculate_readmission_risk(self, demographics, comorbidities, clinical,
                                   utilization, encounter, sdoh):
        """
        Calculate 30-day readmission probability based on literature-validated risk factors.

        Key predictors from clinical literature:
        - Prior readmissions (strongest predictor)
        - Charlson score (comorbidity burden)
        - Length of stay (U-shaped: very short or very long)
        - Discharge disposition (SNF/Rehab higher risk)
        - Social determinants (housing, food, transportation)
        - Lab abnormalities (creatinine, hemoglobin)
        - Recent utilization (ED visits, hospitalizations)
        - Poor care transitions (no follow-up, medication changes)
        """
        n = len(demographics)

        # Start with base rate (~15%)
        risk_score = np.ones(n) * 0.15

        # Prior readmissions (strongest predictor: +8% per readmission)
        risk_score += 0.08 * utilization['prior_readmissions']

        # Charlson score (+3% per point)
        risk_score += 0.03 * comorbidities['charlson_score']

        # Age (65+ = +5%, 75+ = +8%)
        risk_score += 0.05 * (demographics['age'] >= 65)
        risk_score += 0.03 * (demographics['age'] >= 75)

        # Length of stay (very short discharge = +10%, very long = +12%)
        risk_score += 0.10 * (encounter['length_of_stay'] <= 2)
        risk_score += 0.12 * (encounter['length_of_stay'] >= 10)

        # ICU admission (+7%)
        risk_score += 0.07 * encounter['icu_admission']

        # Discharge to SNF/Rehab (not home = +9%)
        risk_score += 0.09 * (~encounter['discharge_disposition'].isin(['Home']))

        # Recent ED visits (+4% per visit, capped)
        risk_score += np.minimum(0.04 * utilization['ed_visits_1yr'], 0.15)

        # Recent hospitalizations (+5% per hospitalization)
        risk_score += np.minimum(0.05 * utilization['hospitalizations_1yr'], 0.20)

        # Poor PCP engagement (0 visits = +8%)
        risk_score += 0.08 * (utilization['pcp_visits_1yr'] == 0)

        # Lab abnormalities
        # High creatinine (>2.0 = +6%)
        risk_score += 0.06 * (clinical['creatinine'] > 2.0)
        # Low hemoglobin (<10 = +5%)
        risk_score += 0.05 * (clinical['hemoglobin'] < 10)
        # Elevated BNP (>500 = +7%)
        risk_score += 0.07 * (clinical['bnp'] > 500)

        # Social determinants
        risk_score += 0.09 * sdoh['housing_unstable']
        risk_score += 0.06 * sdoh['food_insecure']
        risk_score += 0.05 * sdoh['transportation_barrier']
        risk_score -= 0.04 * sdoh['has_caregiver']  # Protective

        # High ADI (>75 = +4%)
        risk_score += 0.04 * (sdoh['adi_score'] > 75)

        # Care transitions
        # No follow-up scheduled (+10%)
        risk_score += 0.10 * (encounter['followup_scheduled'] == 0)
        # Many medication changes (+3%)
        risk_score += 0.03 * (encounter['medication_changes'] >= 5)

        # Polypharmacy (>15 meds = +5%)
        risk_score += 0.05 * (encounter['num_medications'] > 15)

        # Insurance (Medicaid/Uninsured = +6%)
        risk_score += 0.06 * demographics['insurance'].isin(['Medicaid', 'Uninsured'])

        # Add random noise
        noise = np.random.normal(0, 0.05, n)
        risk_score += noise

        # Clip to valid probability range
        risk_score = np.clip(risk_score, 0.01, 0.95)

        return risk_score

    def introduce_missing_data(self, df, missing_rate=0.23):
        """
        Introduce realistic missing data patterns (23% overall missingness).

        Missing patterns:
        - MCAR (Missing Completely at Random): Random missingness
        - MAR (Missing at Random): Correlated with other variables
        - MNAR (Missing Not at Random): Related to the missing value itself
        """
        df_copy = df.copy()

        # Define missingness rates by variable type
        high_missing = ['adi_score', 'days_since_last_hosp', 'bnp', 'hba1c']  # 30-40%
        medium_missing = ['bmi', 'oxygen_saturation', 'pcp_visits_1yr']  # 15-25%
        low_missing = ['creatinine', 'hemoglobin', 'systolic_bp']  # 5-10%

        # Introduce missingness
        for col in df_copy.columns:
            if col in high_missing:
                mask = np.random.random(len(df_copy)) < 0.35
                df_copy.loc[mask, col] = np.nan
            elif col in medium_missing:
                mask = np.random.random(len(df_copy)) < 0.20
                df_copy.loc[mask, col] = np.nan
            elif col in low_missing:
                mask = np.random.random(len(df_copy)) < 0.08
                df_copy.loc[mask, col] = np.nan

        return df_copy

    def generate(self):
        """Generate complete synthetic EHR dataset."""
        print(f"Generating synthetic EHR data for {self.n_patients:,} patients...")

        # Generate each component
        demographics = self.generate_demographics()
        print("  ✓ Demographics generated")

        comorbidities = self.generate_comorbidities(demographics)
        print("  ✓ Comorbidities generated")

        clinical = self.generate_clinical_data(demographics, comorbidities)
        print("  ✓ Clinical data (labs, vitals) generated")

        utilization = self.generate_utilization_history(demographics, comorbidities)
        print("  ✓ Utilization history generated")

        encounter = self.generate_current_encounter(demographics, comorbidities, utilization)
        print("  ✓ Current encounter data generated")

        sdoh = self.generate_social_determinants(demographics)
        print("  ✓ Social determinants of health generated")

        # Generate discharge summaries
        print("  Generating discharge summaries...")
        discharge_summaries = []
        for i in range(self.n_patients):
            summary = self.generate_discharge_summary(i, demographics, comorbidities, encounter)
            discharge_summaries.append(summary)

        # Calculate readmission risk and labels
        readmission_prob = self.calculate_readmission_risk(
            demographics, comorbidities, clinical, utilization, encounter, sdoh
        )

        readmission_30d = (np.random.random(self.n_patients) < readmission_prob).astype(int)

        print(f"  ✓ Readmission labels generated (base rate: {readmission_30d.mean():.1%})")

        # Combine all data
        df = pd.concat([
            demographics,
            comorbidities,
            clinical,
            utilization,
            encounter,
            sdoh
        ], axis=1)

        # Add discharge summaries and target
        df['discharge_summary'] = discharge_summaries
        df['readmission_30d'] = readmission_30d

        # Introduce realistic missing data patterns
        df = self.introduce_missing_data(df)

        missing_pct = df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100
        print(f"  ✓ Missing data introduced ({missing_pct:.1f}% overall)")

        print(f"\nDataset complete: {len(df):,} patients, {len(df.columns)} features")

        return df

# Generate the data
generator = HealthcareDataGenerator(n_patients=10000, random_state=42)
df = generator.generate()

## 2. Data Exploration

In [ ]:
# Dataset overview
print("Dataset Shape:", df.shape)
print(f"\nReadmission Rate: {df['readmission_30d'].mean():.2%}")
print(f"Total Readmissions: {df['readmission_30d'].sum():,}")

# Missing data summary
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_summary = missing_summary[missing_summary > 0]
print(f"\nColumns with Missing Data:")
for col, count in missing_summary.head(10).items():
    print(f"  {col}: {count:,} ({count/len(df)*100:.1f}%)")

In [ ]:
# Demographics breakdown
print("\n=== DEMOGRAPHICS ===")
print("\nAge Distribution:")
print(df['age'].describe())

print("\nRace Distribution:")
print(df['race'].value_counts())
print("\nRace Proportions:")
print(df['race'].value_counts(normalize=True))

print("\nInsurance Distribution:")
print(df['insurance'].value_counts())

print("\nAge Group Distribution:")
print(df['age_group'].value_counts().sort_index())

In [ ]:
# Clinical characteristics
print("\n=== CLINICAL CHARACTERISTICS ===")
print("\nCharlson Comorbidity Index:")
print(df['charlson_score'].describe())

print("\nElixhauser Score:")
print(df['elixhauser_score'].describe())

print("\nComorbidity Prevalence:")
comorbidity_cols = ['diabetes', 'chf', 'copd', 'ckd', 'cancer', 'dementia',
                    'hypertension', 'depression', 'obesity', 'anemia']
for col in comorbidity_cols:
    if col in df.columns:
        prev = df[col].mean() * 100
        print(f"  {col}: {prev:.1f}%")

In [ ]:
# Readmission rates by demographic groups (fairness check)
print("\n=== READMISSION RATES BY DEMOGRAPHIC GROUP ===")

print("\nBy Race:")
race_readm = df.groupby('race')['readmission_30d'].agg(['mean', 'count'])
race_readm.columns = ['Readmission Rate', 'N']
race_readm['Readmission Rate'] = race_readm['Readmission Rate'] * 100
print(race_readm.sort_values('Readmission Rate', ascending=False))

print("\nBy Age Group:")
age_readm = df.groupby('age_group')['readmission_30d'].agg(['mean', 'count'])
age_readm.columns = ['Readmission Rate', 'N']
age_readm['Readmission Rate'] = age_readm['Readmission Rate'] * 100
print(age_readm)

print("\nBy Insurance:")
ins_readm = df.groupby('insurance')['readmission_30d'].agg(['mean', 'count'])
ins_readm.columns = ['Readmission Rate', 'N']
ins_readm['Readmission Rate'] = ins_readm['Readmission Rate'] * 100
print(ins_readm.sort_values('Readmission Rate', ascending=False))

In [ ]:
# Visualize key patterns
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Age distribution
axes[0, 0].hist(df['age'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Age Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Age (years)')
axes[0, 0].set_ylabel('Frequency')

# Readmission by Charlson score
charlson_readm = df.groupby('charlson_score')['readmission_30d'].mean() * 100
axes[0, 1].bar(charlson_readm.index, charlson_readm.values, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Readmission Rate by Charlson Score', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Charlson Comorbidity Index')
axes[0, 1].set_ylabel('Readmission Rate (%)')

# Readmission by race
race_order = race_readm.sort_values('Readmission Rate', ascending=False).index
race_data = df.groupby('race')['readmission_30d'].mean() * 100
axes[1, 0].bar(range(len(race_data)), race_data[race_order].values,
               tick_label=race_order, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Readmission Rate by Race', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Readmission Rate (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Length of stay distribution
axes[1, 1].hist(df['length_of_stay'], bins=30, edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Length of Stay Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Length of Stay (days)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Sample discharge summaries
print("\n=== SAMPLE DISCHARGE SUMMARIES ===")
print("\nHigh-Risk Patient (Readmitted):")
high_risk = df[df['readmission_30d'] == 1].sample(1, random_state=42)
print(f"Age: {high_risk['age'].values[0]}, Race: {high_risk['race'].values[0]}, "
      f"Insurance: {high_risk['insurance'].values[0]}")
print(f"Charlson: {high_risk['charlson_score'].values[0]}, "
      f"Prior readmissions: {high_risk['prior_readmissions'].values[0]}")
print(f"\nNote: {high_risk['discharge_summary'].values[0]}")

print("\n" + "="*80)
print("\nLow-Risk Patient (Not Readmitted):")
low_risk = df[df['readmission_30d'] == 0].sample(1, random_state=43)
print(f"Age: {low_risk['age'].values[0]}, Race: {low_risk['race'].values[0]}, "
      f"Insurance: {low_risk['insurance'].values[0]}")
print(f"Charlson: {low_risk['charlson_score'].values[0]}, "
      f"Prior readmissions: {low_risk['prior_readmissions'].values[0]}")
print(f"\nNote: {low_risk['discharge_summary'].values[0]}")

## 3. Data Preparation and Train/Test Split

We'll split the data while preserving:
1. **Temporal integrity**: Simulate time-based split (train on earlier patients)
2. **Stratification**: Ensure demographic balance across train/test
3. **Protected attributes**: Keep race, age_group, insurance for fairness testing

In [ ]:
# Separate features from target and protected attributes
protected_attrs = ['race', 'age_group', 'insurance']
target = 'readmission_30d'
text_feature = 'discharge_summary'

# Create stratification variable (race + readmission)
df['strat_var'] = df['race'].astype(str) + '_' + df['readmission_30d'].astype(str)

# Train/test split (80/20) with stratification
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['strat_var']
)

# Further split train into train/val for calibration
train_df, val_df = train_test_split(
    train_df,
    test_size=0.15,
    random_state=42,
    stratify=train_df['strat_var']
)

print(f"Train set: {len(train_df):,} patients ({train_df['readmission_30d'].mean():.2%} readmission)")
print(f"Validation set: {len(val_df):,} patients ({val_df['readmission_30d'].mean():.2%} readmission)")
print(f"Test set: {len(test_df):,} patients ({test_df['readmission_30d'].mean():.2%} readmission)")

# Verify demographic balance
print("\nRace distribution across splits:")
print("Train:")
print(train_df['race'].value_counts(normalize=True).sort_index())
print("\nTest:")
print(test_df['race'].value_counts(normalize=True).sort_index())

## 4. HIPAA-Compliant Data Pipeline

Implement Safe Harbor de-identification method:
- Remove 18 identifiers (already done in synthetic data)
- Age aggregation (85+ capped)
- Date shifting (not applicable for synthetic data)
- ZIP code generalization (not included)

In production, this would include:
- PHI scrubbing from clinical notes
- Encryption at rest and in transit
- Audit logging
- Access controls

In [ ]:
def apply_safe_harbor_deidentification(df):
    """
    Apply HIPAA Safe Harbor de-identification.

    Note: Synthetic data is already de-identified, but this demonstrates
    the de-identification process for real PHI.
    """
    df_deidentified = df.copy()

    # Age capping (HIPAA requires aggregating 90+ or 85+)
    df_deidentified['age_deidentified'] = np.where(
        df_deidentified['age'] >= 90,
        90,  # Cap at 90
        df_deidentified['age']
    )

    # In production, would also:
    # - Remove patient names, MRNs, account numbers
    # - Scrub dates (replace with offsets)
    # - Remove precise geographic data (keep state only)
    # - Scrub clinical notes for remaining PHI

    print("✓ Safe Harbor de-identification applied")
    print(f"  - Ages 90+ capped: {(df['age'] >= 90).sum()} patients")

    return df_deidentified

# Apply de-identification
train_df = apply_safe_harbor_deidentification(train_df)
val_df = apply_safe_harbor_deidentification(val_df)
test_df = apply_safe_harbor_deidentification(test_df)

## 5. Baseline Model: LACE Index

The LACE index is a validated clinical tool for predicting readmission:
- **L**ength of stay (0-7 points)
- **A**cute admission (3 points)
- **C**harlson score (0-5 points)
- **E**D visits in past 6 months (0-4 points)

Total score: 0-19 (higher = higher risk)
Threshold: ≥10 for high risk

In [ ]:
def calculate_lace_index(df):
    """
    Calculate LACE index for readmission prediction.

    This is the clinical baseline that AutoGluon will be compared against.
    """
    lace_score = np.zeros(len(df))

    # L: Length of stay
    # 1-2 days: 1pt, 3 days: 2pts, 4-6 days: 3pts, 7-13 days: 4pts, 14+ days: 5pts
    los = df['length_of_stay'].fillna(0)
    lace_score += np.where(los < 1, 0,
                  np.where(los <= 2, 1,
                  np.where(los == 3, 2,
                  np.where(los <= 6, 3,
                  np.where(los <= 13, 4, 5)))))

    # A: Acute admission (Emergency admission = 3 points)
    lace_score += 3 * (df['admission_source'] == 'Emergency').astype(int)

    # C: Charlson score (capped at 5)
    lace_score += np.minimum(df['charlson_score'].fillna(0), 5)

    # E: ED visits in past 6 months
    # 0: 0pts, 1: 1pt, 2: 2pts, 3: 3pts, 4+: 4pts
    ed_visits = df['ed_visits_1yr'].fillna(0) / 2  # Approximate 6-month from annual
    lace_score += np.minimum(ed_visits, 4)

    return lace_score

# Calculate LACE for all sets
train_df['lace_index'] = calculate_lace_index(train_df)
val_df['lace_index'] = calculate_lace_index(val_df)
test_df['lace_index'] = calculate_lace_index(test_df)

# Evaluate LACE index on test set (threshold ≥10 for high risk)
lace_predictions = (test_df['lace_index'] >= 10).astype(int)
lace_scores = test_df['lace_index'] / 19  # Normalize to probability scale

lace_auc = roc_auc_score(test_df['readmission_30d'], lace_scores)
lace_cm = confusion_matrix(test_df['readmission_30d'], lace_predictions)

print("\n=== LACE INDEX BASELINE PERFORMANCE ===")
print(f"AUC: {lace_auc:.3f}")
print(f"\nConfusion Matrix (threshold ≥10):")
print(lace_cm)

tn, fp, fn, tp = lace_cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print(f"\nSensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

print(f"\nLACE Index Summary:")
print(f"  Mean: {test_df['lace_index'].mean():.2f}")
print(f"  Median: {test_df['lace_index'].median():.2f}")
print(f"  Range: {test_df['lace_index'].min():.0f} - {test_df['lace_index'].max():.0f}")

## 6. Feature Engineering

Prepare features for AutoGluon:
1. **Structured features**: Demographics, comorbidities, labs, vitals, utilization, encounter, SDOH
2. **Text features**: Discharge summaries (will use MultiModalPredictor)
3. **Missing indicators**: Flag missingness patterns
4. **Interaction features**: Comorbidity × age, etc.

In [ ]:
def engineer_features(df):
    """
    Create engineered features for AutoGluon.
    """
    df_feat = df.copy()

    # Missing indicators (important for EHR data)
    important_vars = ['bnp', 'hba1c', 'creatinine', 'hemoglobin', 'adi_score']
    for var in important_vars:
        if var in df_feat.columns:
            df_feat[f'{var}_missing'] = df_feat[var].isnull().astype(int)

    # Interaction features
    # Age × comorbidity burden
    df_feat['age_charlson_interaction'] = df_feat['age'] * df_feat['charlson_score']

    # Social vulnerability score
    df_feat['social_vuln_score'] = (
        df_feat['housing_unstable'].fillna(0) +
        df_feat['food_insecure'].fillna(0) +
        df_feat['transportation_barrier'].fillna(0) +
        (1 - df_feat['has_caregiver'].fillna(0))
    )

    # Care transitions risk score
    df_feat['care_transitions_risk'] = (
        (1 - df_feat['followup_scheduled'].fillna(0)) * 3 +
        np.minimum(df_feat['medication_changes'].fillna(0) / 5, 2)
    )

    # Lab abnormality flags
    df_feat['creatinine_high'] = (df_feat['creatinine'] > 2.0).astype(int)
    df_feat['hemoglobin_low'] = (df_feat['hemoglobin'] < 10).astype(int)
    df_feat['bnp_high'] = (df_feat['bnp'] > 500).astype(int)

    # Very short or very long LOS (both risky)
    df_feat['los_very_short'] = (df_feat['length_of_stay'] <= 2).astype(int)
    df_feat['los_very_long'] = (df_feat['length_of_stay'] >= 10).astype(int)

    # Polypharmacy flag
    df_feat['polypharmacy'] = (df_feat['num_medications'] > 15).astype(int)

    return df_feat

# Apply feature engineering
train_df_feat = engineer_features(train_df)
val_df_feat = engineer_features(val_df)
test_df_feat = engineer_features(test_df)

print(f"Features engineered. Total features: {len(train_df_feat.columns)}")

## 7. Multimodal Learning: Text Embeddings from Discharge Summaries

We'll use AutoGluon's **MultiModalPredictor** to:
1. Pre-train on discharge summaries
2. Extract 64-dimensional text embeddings
3. Combine with structured features for final model

This captures clinical patterns in unstructured text (e.g., "poor social support", "non-compliant with medications")

In [ ]:
from autogluon.multimodal import MultiModalPredictor

# Prepare text data
text_train = train_df_feat[['discharge_summary', 'readmission_30d']].copy()
text_val = val_df_feat[['discharge_summary', 'readmission_30d']].copy()
text_test = test_df_feat[['discharge_summary', 'readmission_30d']].copy()

print("Training MultiModalPredictor on discharge summaries...")
print(f"Training samples: {len(text_train):,}")
print("\nThis may take 5-10 minutes...\n")

# Train text model (lightweight config for notebook)


# Train with default settings - AutoGluon will automatically detect text columns
text_predictor = MultiModalPredictor(
    label='readmission_30d',
    problem_type='binary',
    eval_metric='roc_auc',
    verbosity=2
)

text_predictor.fit(
    train_data=text_train,
    tuning_data=text_val,
    time_limit=300  # 5 minutes
)


In [ ]:
# AutoGluon has built-in evaluation that handles this automatically
text_scores = text_predictor.evaluate(text_test)
print(f"Text model ROC-AUC: {text_scores['roc_auc']:.4f}")

# You can also get multiple metrics at once
text_predictor.evaluate(text_test, metrics=['roc_auc', 'accuracy', 'f1'])


In [ ]:
# Step 1: Extract embeddings and convert to DataFrames
print("Step 1: Extracting text embeddings...")
train_emb_array = text_predictor.extract_embedding(text_train)
val_emb_array = text_predictor.extract_embedding(text_val)
test_emb_array = text_predictor.extract_embedding(text_test)

# Convert to DataFrames
train_embeddings = pd.DataFrame(
    train_emb_array,
    columns=[f'text_emb_{i}' for i in range(train_emb_array.shape[1])],
    index=train_df_feat.index  # Use original index
)

val_embeddings = pd.DataFrame(
    val_emb_array,
    columns=[f'text_emb_{i}' for i in range(val_emb_array.shape[1])],
    index=val_df_feat.index
)

test_embeddings = pd.DataFrame(
    test_emb_array,
    columns=[f'text_emb_{i}' for i in range(test_emb_array.shape[1])],
    index=test_df_feat.index
)

print(f"✓ Embeddings shape: {train_embeddings.shape}")

# Step 2: Combine embeddings with structured features
print("\nStep 2: Combining with structured features...")

# Get structured features (excluding text and label)
feature_cols = [col for col in train_df_feat.columns
                if col not in ['discharge_summary', 'readmission_30d']]

# Combine features
train_combined = pd.concat([
    train_df_feat[feature_cols],
    train_embeddings,
    train_df_feat[['readmission_30d']]  # Add label at the end
], axis=1)

val_combined = pd.concat([
    val_df_feat[feature_cols],
    val_embeddings,
    val_df_feat[['readmission_30d']]
], axis=1)

test_combined = pd.concat([
    test_df_feat[feature_cols],
    test_embeddings,
    test_df_feat[['readmission_30d']]
], axis=1)

print(f"✓ Combined features shape: {train_combined.shape}")
print(f"  - Structured features: {len(feature_cols)}")
print(f"  - Text embeddings: {train_embeddings.shape[1]}")
print(f"  - Total features: {train_combined.shape[1] - 1}")  # -1 for label



## 8. Prepare Final Feature Sets

Combine structured features + text embeddings for multimodal model

In [ ]:
# Define features to exclude
exclude_features = [
    'readmission_30d',  # Target
    'discharge_summary',  # Text (already embedded)
    'strat_var',  # Utility variable
    'age_group',  # Keep for fairness testing (not as feature)
    'race',  # Protected attribute (not as feature)
    'insurance',  # Protected attribute (not as feature)
    'gender',  # Categorical, low cardinality - could keep but excluding for simplicity
    'marital_status',  # Categorical
    'admission_source',  # Categorical
    'discharge_disposition'  # Categorical
]

# Get structured feature columns
structured_features = [
    col for col in train_df_feat.columns
    if col not in exclude_features
]

print(f"Structured features: {len(structured_features)}")
print(f"Text embedding features: {len(train_embeddings.columns)}")
print(f"Total features for multimodal model: {len(structured_features) + len(train_embeddings.columns)}")

# Combine structured + text embeddings
train_combined = pd.concat([
    train_df_feat[structured_features],
    train_embeddings
], axis=1)
train_combined['readmission_30d'] = train_df_feat['readmission_30d'].values

val_combined = pd.concat([
    val_df_feat[structured_features],
    val_embeddings
], axis=1)
val_combined['readmission_30d'] = val_df_feat['readmission_30d'].values

test_combined = pd.concat([
    test_df_feat[structured_features],
    test_embeddings
], axis=1)
test_combined['readmission_30d'] = test_df_feat['readmission_30d'].values

print("\n✓ Multimodal feature sets prepared")
print(f"  Train: {train_combined.shape}")
print(f"  Val: {val_combined.shape}")
print(f"  Test: {test_combined.shape}")

In [ ]:
# Common leakage sources in medical data
suspicious_cols = [
    'readmission_days',  # If this exists - direct leakage
    'readmitted',        # Alternative label column
    'discharge_disposition',  # May contain readmission info
    'patient_id',        # Should not be used for prediction
    'encounter_id',      # Should not be used
    'admission_id'       # Should not be used
]

# Remove any suspicious columns that exist
cols_to_remove = [col for col in suspicious_cols if col in train_combined.columns]
if cols_to_remove:
    print(f"Removing suspicious columns: {cols_to_remove}")
    train_combined_clean = train_combined.drop(columns=cols_to_remove)
    val_combined_clean = val_combined.drop(columns=cols_to_remove)
    test_combined_clean = test_combined.drop(columns=cols_to_remove)
else:
    train_combined_clean = train_combined
    val_combined_clean = val_combined
    test_combined_clean = test_combined

# Also check text embedding dimensions - they might be memorizing
print(f"\nText embedding dimensions: {sum(1 for col in train_combined.columns if 'text_emb_' in col)}")

In [ ]:
from autogluon.tabular import TabularPredictor

# Retrain with more conservative settings
clean_predictor = TabularPredictor(
    label='readmission_30d',
    eval_metric='roc_auc',
    verbosity=2
)

# Use hyperparameters that reduce overfitting
hyperparameters = {
    'GBM': [
        {
            'num_boost_round': 100,
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.5,
            'bagging_fraction': 0.5,
            'bagging_freq': 5,
            'min_data_in_leaf': 20,
            'lambda_l1': 1.0,
            'lambda_l2': 1.0,
        }
    ],
    'RF': [
        {
            'n_estimators': 100,
            'max_depth': 10,
            'min_samples_leaf': 20,
            'min_samples_split': 40,
            'max_features': 'sqrt',
        }
    ],
    'NN_TORCH': [
        {
            'num_epochs': 10,
            'learning_rate': 0.0001,
            'dropout_prob': 0.5,
            'hidden_dim': 64,
            'num_layers': 2,
        }
    ],
}

clean_predictor.fit(
    train_data=train_combined_clean,
    tuning_data=val_combined_clean,
    hyperparameters=hyperparameters,
    time_limit=300,
    presets='optimize_for_deployment'  # More conservative
)

# Evaluate
clean_scores = clean_predictor.evaluate(test_combined_clean)
print(f"\nCleaned model AUC: {clean_scores['roc_auc']:.4f}")

In [ ]:
# Remove the problematic strat_var feature
print("Removing data leakage source: strat_var")

# Check if strat_var exists in the data
if 'strat_var' in train_combined.columns:
    print(f"Found strat_var in features - this is causing the perfect score!")

    # Remove it from all datasets
    train_clean = train_combined.drop(columns=['strat_var'])
    val_clean = val_combined.drop(columns=['strat_var'])
    test_clean = test_combined.drop(columns=['strat_var'])
else:
    print("strat_var not found directly, checking variations...")
    # Check for similar names
    strat_cols = [col for col in train_combined.columns if 'strat' in col.lower()]
    print(f"Found stratification-related columns: {strat_cols}")

    train_clean = train_combined.drop(columns=strat_cols, errors='ignore')
    val_clean = val_combined.drop(columns=strat_cols, errors='ignore')
    test_clean = test_combined.drop(columns=strat_cols, errors='ignore')

# Also remove discharge_disposition if it exists
cols_to_remove = ['strat_var', 'discharge_disposition']
for col in cols_to_remove:
    if col in train_clean.columns:
        train_clean = train_clean.drop(columns=[col])
        val_clean = val_clean.drop(columns=[col])
        test_clean = test_clean.drop(columns=[col])
        print(f"Removed: {col}")

print(f"\nCleaned dataset shape: {train_clean.shape}")
print(f"Features remaining: {train_clean.shape[1] - 1}")  # -1 for label

In [ ]:
from autogluon.tabular import TabularPredictor

# Train with cleaned data
final_predictor = TabularPredictor(
    label='readmission_30d',
    eval_metric='roc_auc',
    path='./ag_final_model',
    verbosity=2
)

# Use standard presets now that leakage is removed
final_predictor.fit(
    train_data=train_clean,
    tuning_data=val_clean,
    time_limit=300,
    presets='medium_quality'
)

# Evaluate
final_scores = final_predictor.evaluate(test_clean)
print(f"\n=== FINAL RESULTS (No Data Leakage) ===")
print(f"Combined Model AUC: {final_scores['roc_auc']:.4f}")

# Get predictions for detailed analysis
final_probs = final_predictor.predict_proba(test_clean)
if isinstance(final_probs, pd.DataFrame):
    final_probs_positive = final_probs.iloc[:, 1]
else:
    final_probs_positive = final_probs[:, 1]

from sklearn.metrics import roc_auc_score, classification_report
final_auc = roc_auc_score(test_clean['readmission_30d'], final_probs_positive)

print(f"\nVerified AUC: {final_auc:.4f}")
print("\nClassification Report:")
print(classification_report(test_clean['readmission_30d'],
                          final_predictor.predict(test_clean)))

In [ ]:
# Check feature importance to identify potential leakage
feature_importance = final_predictor.feature_importance(test_combined)
top_features = feature_importance.head(20)
print("Top 20 Most Important Features:")
print(top_features)

# Look for suspiciously high importance features
suspicious_features = feature_importance[feature_importance['importance'] > 0.1]
print(f"\nHighly Important Features (>10% importance):")
print(suspicious_features)

In [ ]:
plt.figure(figsize=(10, 8))
# Use the index for feature names and 'importance' column for bar width
plt.barh(top_features.index, top_features['importance'])
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Top 20 Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis to show most important at top
plt.show()

In [ ]:
# Check correlation with target variable
import numpy as np
from scipy.stats import pearsonr

# Get all feature columns (excluding label)
feature_cols = [col for col in train_combined.columns if col != 'readmission_30d']

# Calculate correlations
correlations = []
for col in feature_cols:
    if train_combined[col].dtype in ['float64', 'int64', 'float32', 'int32']:
        corr, _ = pearsonr(train_combined[col].fillna(0),
                          train_combined['readmission_30d'])
        correlations.append({'feature': col, 'correlation': abs(corr)})

# Find suspiciously high correlations
corr_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)
print("Features with highest correlation to readmission:")
print(corr_df.head(20))

# Flag potential leakage
leakage_candidates = corr_df[corr_df['correlation'] > 0.8]
if not leakage_candidates.empty:
    print(f"\n⚠️ WARNING: Potential data leakage detected!")
    print(f"Features with correlation > 0.8:")
    print(leakage_candidates)

In [ ]:
# Feature importance for clean model
if hasattr(final_predictor, 'feature_importance'):
    clean_importance = final_predictor.feature_importance(test_clean)
    print("\nTop 10 Important Features (Clean Model):")
    print(clean_importance.head(10))

In [ ]:
# Double-check by looking at feature importance distribution
importance_values = clean_importance['importance'].values
max_importance = importance_values.max()

if max_importance > 0.3:
    print(f"\n⚠️ Warning: Feature '{clean_importance.iloc[0]['feature']}' has {max_importance:.2%} importance")
    print("This might indicate remaining leakage")
else:
    print(f"\n✓ Maximum feature importance: {max_importance:.2%}")
    print("Feature importance distribution looks reasonable")

# Check model performance metrics are realistic
print(f"\nModel performance sanity check:")
print(f"AUC between 0.65-0.85: {'✓' if 0.65 <= final_auc <= 0.85 else '❌'}")

## 9. Fairness-Aware Model Training

We'll train AutoGluon with fairness-aware techniques:

### Technique 1: Sample Re-weighting
Upweight underrepresented groups to balance class distributions across demographics

### Technique 2: Stratified Ensembling
Train separate models for each demographic group and ensemble

### Technique 3: Post-processing Calibration
Calibrate predictions separately by group to ensure equal TPR/FPR

In [ ]:
# Technique 1: Sample re-weighting by race
# Calculate weights to balance readmission rates across race groups

race_counts = train_df_feat['race'].value_counts()
majority_count = race_counts.max()

# Weight inversely proportional to group size
race_weights = train_df_feat['race'].map(
    lambda r: majority_count / race_counts[r]
)

train_combined['sample_weight'] = race_weights.values

print("Sample weights by race:")
for race in race_counts.index:
    weight = majority_count / race_counts[race]
    print(f"  {race}: {weight:.3f}")

In [ ]:


# Keep validation data separate with use_bag_holdout
from autogluon.tabular import TabularPredictor

print("Training final model with bag holdout...")
final_predictor = TabularPredictor(
    label='readmission_30d',
    problem_type='binary',
    eval_metric='roc_auc',
    path='./ag_final_model',
    verbosity=2
)

final_predictor.fit(
    train_data=train_clean,
    tuning_data=val_clean,
    time_limit=600,
    presets='best_quality',
    use_bag_holdout=True  # This allows using tuning_data with bagged mode
)

print("\n✓ Model training complete")

In [ ]:
# Evaluate the model
print("\nEvaluating model performance...")
final_scores = final_predictor.evaluate(test_clean)

# Get detailed predictions
final_probs = final_predictor.predict_proba(test_clean)
if isinstance(final_probs, pd.DataFrame):
    final_probs_positive = final_probs.iloc[:, 1]
else:
    final_probs_positive = final_probs[:, 1]

from sklearn.metrics import roc_auc_score, classification_report
final_auc = roc_auc_score(test_clean['readmission_30d'], final_probs_positive)

print("\n" + "="*60)
print("FINAL RESULTS (Clean Model)")
print("="*60)
print(f"ROC-AUC Score: {final_auc:.4f}")
print(f"\nModel Comparison:")
print(f"  LACE Baseline:     {lace_auc:.4f}")

print(f"  Combined (clean):  {final_auc:.4f}")
print(f"\nImprovement over baseline: +{(final_auc - lace_auc):.4f}")

## 10. Model Evaluation: Performance Metrics

In [ ]:
# If you have a successful model from earlier
from autogluon.tabular import TabularPredictor

# Use the model trained earlier in the notebook
model_path = "./ag_final_model"

if os.path.exists(model_path):
    predictor = TabularPredictor.load(model_path)
    print(f"Loaded model from: {model_path}")
else:
    # Use the final_predictor trained earlier in this notebook
    print(f"Model not found at {model_path}, using final_predictor from earlier training")
    predictor = final_predictor

In [ ]:
# Get predictions on test set
test_preds_proba = predictor.predict_proba(test_clean)
if isinstance(test_preds_proba, pd.DataFrame):
    test_preds_proba = test_preds_proba[1]  # Get positive class probabilities

test_preds = predictor.predict(test_clean)

# Overall performance
test_auc = roc_auc_score(test_clean['readmission_30d'], test_preds_proba)
test_cm = confusion_matrix(test_clean['readmission_30d'], test_preds)
brier = brier_score_loss(test_clean['readmission_30d'], test_preds_proba)

print("\n" + "="*60)
print("=== OVERALL PERFORMANCE ===")
print("="*60)
print(f"\nAUC: {test_auc:.4f}")
print(f"Brier Score: {brier:.4f}")
print(f"\nConfusion Matrix:")
print(test_cm)

tn, fp, fn, tp = test_cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print(f"\nSensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"PPV (Precision): {ppv:.3f}")
print(f"NPV: {npv:.3f}")

print(f"\n{'='*60}")
print("COMPARISON TO BASELINE")
print(f"{'='*60}")
print(f"AutoGluon AUC: {test_auc:.4f}")
print(f"LACE Index AUC: {lace_auc:.4f}")
print(f"Improvement: {(test_auc - lace_auc):.4f} ({(test_auc - lace_auc)/lace_auc*100:.1f}%)")

In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(test_combined['readmission_30d'], test_preds_proba)
lace_fpr, lace_tpr, _ = roc_curve(test_df['readmission_30d'], lace_scores)

plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, label=f'AutoGluon (AUC = {test_auc:.3f})', linewidth=2)
plt.plot(lace_fpr, lace_tpr, label=f'LACE Index (AUC = {lace_auc:.3f})', linewidth=2, linestyle='--')
plt.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve: AutoGluon vs. LACE Index', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calibration curve
fig, ax = plt.subplots(1, 1, figsize=(10, 7))

fraction_of_positives, mean_predicted_value = calibration_curve(
    test_combined['readmission_30d'],
    test_preds_proba,
    n_bins=10
)

ax.plot(mean_predicted_value, fraction_of_positives, "s-", linewidth=2, label='AutoGluon')
ax.plot([0, 1], [0, 1], "k--", label="Perfectly calibrated")
ax.set_xlabel('Mean Predicted Probability', fontsize=12)
ax.set_ylabel('Fraction of Positives', fontsize=12)
ax.set_title('Calibration Curve', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nCalibration Assessment:")
print(f"  Brier score: {brier:.4f} (lower is better, perfect = 0)")
print(f"  Expected Calibration Error: ~{np.mean(np.abs(fraction_of_positives - mean_predicted_value)):.4f}")

## 11. Fairness Evaluation

Evaluate fairness across protected demographic groups:
- **Race**: White, Black, Hispanic, Asian, Other
- **Age**: 18-44, 45-64, 65-74, 75+
- **Insurance**: Medicare, Commercial, Medicaid, Uninsured

### Fairness Metrics
1. **Group AUC**: Performance should be similar across groups (gap <5pp)
2. **Equalized Odds**: TPR and FPR should be similar across groups
3. **Calibration**: Predicted probabilities should match actual rates in each group
4. **Demographic Parity**: Positive prediction rates should be similar

In [ ]:
# Add predictions to test dataframe for fairness analysis
test_df_fair = test_df_feat.copy()
test_df_fair['predicted_prob'] = test_preds_proba.values
test_df_fair['predicted_class'] = test_preds.values
test_df_fair['actual'] = test_clean['readmission_30d'].values

In [ ]:
def calculate_fairness_metrics(df, group_col, pred_col='predicted_prob',
                               pred_class_col='predicted_class', actual_col='actual'):
    """
    Calculate fairness metrics for each group.

    Returns:
    - AUC by group
    - TPR (sensitivity) by group
    - FPR (1 - specificity) by group
    - Calibration metrics by group
    - Positive rate by group
    """
    results = []

    for group in df[group_col].unique():
        if pd.isna(group):
            continue

        group_df = df[df[group_col] == group]

        if len(group_df) < 50:  # Skip very small groups
            continue

        # AUC
        try:
            auc = roc_auc_score(group_df[actual_col], group_df[pred_col])
        except:
            auc = np.nan

        # Confusion matrix metrics
        cm = confusion_matrix(group_df[actual_col], group_df[pred_class_col])
        tn, fp, fn, tp = cm.ravel()

        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # Sensitivity
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # 1 - Specificity

        # Positive prediction rate (demographic parity)
        pos_rate = group_df[pred_class_col].mean()

        # Actual positive rate
        actual_pos_rate = group_df[actual_col].mean()

        # Calibration (predicted vs actual)
        mean_pred = group_df[pred_col].mean()
        calibration_error = abs(mean_pred - actual_pos_rate)

        results.append({
            'Group': group,
            'N': len(group_df),
            'Actual_Rate': actual_pos_rate,
            'AUC': auc,
            'TPR': tpr,
            'FPR': fpr,
            'Pos_Pred_Rate': pos_rate,
            'Mean_Pred_Prob': mean_pred,
            'Calibration_Error': calibration_error
        })

    return pd.DataFrame(results)

# Calculate fairness metrics for each protected attribute
print("\n" + "="*80)
print("=== FAIRNESS EVALUATION ===")
print("="*80)

print("\n--- BY RACE ---")
race_metrics = calculate_fairness_metrics(test_df_fair, 'race')
print(race_metrics.to_string(index=False))

print("\n--- BY AGE GROUP ---")
age_metrics = calculate_fairness_metrics(test_df_fair, 'age_group')
print(age_metrics.to_string(index=False))

print("\n--- BY INSURANCE ---")
insurance_metrics = calculate_fairness_metrics(test_df_fair, 'insurance')
print(insurance_metrics.to_string(index=False))

In [ ]:
# Calculate fairness gaps (max - min)
print("\n" + "="*80)
print("FAIRNESS GAPS (Max - Min across groups)")
print("="*80)

print("\nRace:")
print(f"  AUC gap: {race_metrics['AUC'].max() - race_metrics['AUC'].min():.4f}")
print(f"  TPR gap: {race_metrics['TPR'].max() - race_metrics['TPR'].min():.4f}")
print(f"  FPR gap: {race_metrics['FPR'].max() - race_metrics['FPR'].min():.4f}")
print(f"  Calibration error (max): {race_metrics['Calibration_Error'].max():.4f}")

print("\nAge Group:")
print(f"  AUC gap: {age_metrics['AUC'].max() - age_metrics['AUC'].min():.4f}")
print(f"  TPR gap: {age_metrics['TPR'].max() - age_metrics['TPR'].min():.4f}")
print(f"  FPR gap: {age_metrics['FPR'].max() - age_metrics['FPR'].min():.4f}")
print(f"  Calibration error (max): {age_metrics['Calibration_Error'].max():.4f}")

print("\nInsurance:")
print(f"  AUC gap: {insurance_metrics['AUC'].max() - insurance_metrics['AUC'].min():.4f}")
print(f"  TPR gap: {insurance_metrics['TPR'].max() - insurance_metrics['TPR'].min():.4f}")
print(f"  FPR gap: {insurance_metrics['FPR'].max() - insurance_metrics['FPR'].min():.4f}")
print(f"  Calibration error (max): {insurance_metrics['Calibration_Error'].max():.4f}")

print("\n" + "="*80)
print("FAIRNESS ASSESSMENT")
print("="*80)
print("Target: AUC gap <5pp, TPR/FPR gap <5pp, Calibration error <3pp")
print("\nResults:")

race_auc_gap = race_metrics['AUC'].max() - race_metrics['AUC'].min()
if race_auc_gap < 0.05:
    print(f"  ✓ Race AUC gap: {race_auc_gap:.4f} (PASS)")
else:
    print(f"  ✗ Race AUC gap: {race_auc_gap:.4f} (FAIL)")

race_tpr_gap = race_metrics['TPR'].max() - race_metrics['TPR'].min()
if race_tpr_gap < 0.05:
    print(f"  ✓ Race TPR gap: {race_tpr_gap:.4f} (PASS)")
else:
    print(f"  ✗ Race TPR gap: {race_tpr_gap:.4f} (FAIL)")

In [ ]:
# Visualize fairness metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# AUC by race
axes[0, 0].bar(range(len(race_metrics)), race_metrics['AUC'].values,
               tick_label=race_metrics['Group'].values, edgecolor='black', alpha=0.7)
axes[0, 0].axhline(y=test_auc, color='r', linestyle='--', label='Overall AUC')
axes[0, 0].set_title('AUC by Race', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('AUC')
axes[0, 0].legend()
axes[0, 0].set_ylim([0.6, 0.8])
axes[0, 0].tick_params(axis='x', rotation=45)

# TPR/FPR by race
x = np.arange(len(race_metrics))
width = 0.35
axes[0, 1].bar(x - width/2, race_metrics['TPR'].values, width, label='TPR',
               edgecolor='black', alpha=0.7)
axes[0, 1].bar(x + width/2, race_metrics['FPR'].values, width, label='FPR',
               edgecolor='black', alpha=0.7)
axes[0, 1].set_title('TPR/FPR by Race (Equalized Odds)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Rate')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(race_metrics['Group'].values, rotation=45)
axes[0, 1].legend()

# AUC by age group
axes[1, 0].bar(range(len(age_metrics)), age_metrics['AUC'].values,
               tick_label=age_metrics['Group'].values, edgecolor='black', alpha=0.7)
axes[1, 0].axhline(y=test_auc, color='r', linestyle='--', label='Overall AUC')
axes[1, 0].set_title('AUC by Age Group', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('AUC')
axes[1, 0].legend()
axes[1, 0].set_ylim([0.6, 0.8])

# Calibration by race
x = np.arange(len(race_metrics))
width = 0.35
axes[1, 1].bar(x - width/2, race_metrics['Actual_Rate'].values, width,
               label='Actual Rate', edgecolor='black', alpha=0.7)
axes[1, 1].bar(x + width/2, race_metrics['Mean_Pred_Prob'].values, width,
               label='Mean Predicted', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Calibration by Race', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Probability')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(race_metrics['Group'].values, rotation=45)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 12. Feature Importance

Understanding which features drive readmission predictions is critical for:
1. **Clinical trust**: Clinicians need to understand WHY a patient is high-risk
2. **Interventions**: Actionable features guide care coordination
3. **Debugging bias**: Identify if model relies on protected attributes indirectly

In [ ]:
# Get feature importance
importance = predictor.feature_importance(test_clean)

print("\n=== TOP 20 MOST IMPORTANT FEATURES ===")
print(importance.head(20))

# Visualize top features
top_n = 15
top_features = importance.head(top_n)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['importance'].values,
         edgecolor='black', alpha=0.7)
plt.yticks(range(len(top_features)), top_features['importance'].index)
plt.xlabel('Importance', fontsize=12)
plt.title(f'Top {top_n} Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 13. Summary and Comparison to Case Study Results

Let's compare our notebook results to the published Case Study 3 outcomes

In [ ]:
print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)

print("\n1. PERFORMANCE METRICS")
print("-" * 80)
print(f"Overall AUC: {test_auc:.4f}")
print(f"Baseline (LACE): {lace_auc:.4f}")
print(f"Improvement: {(test_auc - lace_auc)/lace_auc*100:.1f}%")
print(f"\nBrier Score: {brier:.4f}")
print(f"Sensitivity: {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

print("\n2. FAIRNESS METRICS")
print("-" * 80)
print(f"Race AUC gap: {(race_metrics['AUC'].max() - race_metrics['AUC'].min())*100:.2f}pp")
print(f"Race TPR gap: {(race_metrics['TPR'].max() - race_metrics['TPR'].min())*100:.2f}pp")
print(f"Race FPR gap: {(race_metrics['FPR'].max() - race_metrics['FPR'].min())*100:.2f}pp")
print(f"Age AUC gap: {(age_metrics['AUC'].max() - age_metrics['AUC'].min())*100:.2f}pp")
print(f"Insurance AUC gap: {(insurance_metrics['AUC'].max() - insurance_metrics['AUC'].min())*100:.2f}pp")

print("\n3. MODEL ARCHITECTURE")
print("-" * 80)
print(f"Total features: {len(train_combined.columns) - 2}")  # -2 for target and sample_weight
print(f"  - Structured features: {len(structured_features)}")
print(f"  - Text embeddings: {len(train_embeddings.columns)}")


print("\n4. COMPARISON TO CASE STUDY 3 TARGETS")
print("-" * 80)
print(f"Target AUC: 0.73")
print(f"Achieved AUC: {test_auc:.3f}")
print(f"Status: {'✓ PASS' if test_auc >= 0.70 else '✗ MISS'}")

print(f"\nTarget fairness gap: <5pp")
race_auc_gap = (race_metrics['AUC'].max() - race_metrics['AUC'].min())
print(f"Achieved race AUC gap: {race_auc_gap*100:.2f}pp")
print(f"Status: {'✓ PASS' if race_auc_gap < 0.05 else '✗ MISS'}")

print("\n" + "="*80)
print("KEY TAKEAWAYS")
print("="*80)
print("")
print("1. Multimodal learning (text + structured data) improves performance")
print("   - Text embeddings capture clinical nuances in discharge summaries")
print("   - Structured features provide quantitative clinical measurements")
print("")
print("2. Fairness-aware training reduces demographic bias")
print("   - Sample re-weighting balances across race groups")
print("   - AUC gaps <5pp demonstrate equitable performance")
print("")
print("3. AutoGluon significantly outperforms clinical baseline (LACE index)")
print(f"   - {(test_auc - lace_auc)/lace_auc*100:.0f}% AUC improvement")
print("   - Better calibration for clinical decision support")
print("")
print("4. Model is production-ready for HIPAA-compliant deployment")
print("   - De-identified data pipeline")
print("   - Fair across protected demographics")
print("   - Interpretable features for clinical trust")
print("\n" + "="*80)

## Next Steps for Production Deployment

This notebook demonstrates the core methodology. For production deployment at HealthFirst Medical Center, the implementation would include:

### 1. Integration
- **Epic EHR Integration**: FHIR API for real-time data extraction
- **Real-time Prediction API**: <2 second latency for clinical workflows
- **Best Practice Advisory (BPA)**: Non-interruptive alerts in Epic

### 2. Monitoring
- **Performance Drift Detection**: Weekly AUC monitoring
- **Fairness Drift Detection**: Monthly demographic performance review
- **Data Quality Monitoring**: Missing data patterns, distribution shifts
- **Alert Fatigue Monitoring**: Override rates, clinician feedback

### 3. Clinical Workflow
- **Care Coordinator Routing**: High-risk patients → Care Transitions program
- **Interpretability**: SHAP values → plain-language clinical explanations
- **Clinician Training**: Model strengths/limitations, appropriate use

### 4. Governance
- **Model Review Committee**: Quarterly performance & fairness audits
- **HIPAA Compliance**: Ongoing audit trail, encryption, access controls
- **Bias Testing**: Semi-annual fairness validation on new data

### 5. Business Impact Tracking
- **Readmission Reduction**: Monthly rates vs. baseline
- **Financial Impact**: Penalty avoidance + cost savings
- **Patient Outcomes**: 30-day mortality, ER revisit rates
- **Clinician Satisfaction**: Survey scores, adoption rates